In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import h5py, os,pathlib, torch, torchani, copy
from torchani.models import ANI1x 
from pathlib import Path

from trainer import *
from sampler import *

/home/shuhao/softwares/miniconda3/envs/acdemo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


you are using /home/shuhao/softwares/miniconda3/envs/acdemo/lib/python3.8/site-packages/torchani


In [4]:
pooldir = Path('/home/shuhao/AMD_demo/pool')

MAX_AL_EPOCH = 2

gpu_list = ["cuda:1"]

model = torchani.models.ANI1x().to(gpu_list[0])
#we will train on relative energy, so disable the energy_shifter in the model
model.energy_shifter.self_energies = torch.tensor([ 0.0, 0.0, 0.0, 0.0]).double().to(gpu_list[0])

for epoch in range(MAX_AL_EPOCH):
    energy_shifter = torchani.utils.EnergyShifter(None)
    training, validation = torchani.data.load(pooldir).subtract_self_energies(energy_shifter).species_to_indices().split(0.8, None)
    training = training.collate(batch_size).cache()
    validation = validation.collate(batch_size).cache()
    model = train(model, training, validation)
    
    new_dataset = pooldir/('ext%d.h5'%(epoch+1))
    #I do not have multiple GPU on this test machine for sampling, so just use the same one
    model_pool = [model]
    for h5file in pooldir.glob('*.h5'):
        _ = dataset_sampler(h5file, model_pool, save_to=str(new_dataset))
    print("%d al epoch done!"%(epoch+1))
    


/home/shuhao/softwares/miniconda3/envs/acdemo/lib/python3.8/site-packages/torchani/resources/
training starting from epoch 1
RMSE: 92.88694609708159 at epoch 2


epoch 1: 100%|██████████| 47/47 [00:04<00:00, 10.24it/s]


best RMSE at this time is:92.887
converged SCF energy = -386.506379264923
extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext1.h5
1 al epoch done!
training starting from epoch 1
RMSE: 76.38035256359292 at epoch 2


epoch 1: 100%|██████████| 47/47 [00:04<00:00,  9.74it/s]


best RMSE at this time is:76.380
converged SCF energy = -386.506192769001
extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext2.h5
converged SCF energy = -386.512895152885
extra sampling done, new datapoints saved to /home/shuhao/AMD_demo/pool/ext2.h5
2 al epoch done!


In [4]:
model

BuiltinModel(
  (aev_computer): AEVComputer(
    (angular_terms): StandardAngular(
      (cutoff_fn): CutoffCosine()
    )
    (radial_terms): StandardRadial(
      (cutoff_fn): CutoffCosine()
    )
    (neighborlist): FullPairwise()
  )
  (neural_networks): Ensemble(
    (0): ANIModel(
      (H): Sequential(
        (0): Linear(in_features=384, out_features=160, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=160, out_features=128, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=128, out_features=96, bias=True)
        (5): CELU(alpha=0.1)
        (6): Linear(in_features=96, out_features=1, bias=True)
      )
      (C): Sequential(
        (0): Linear(in_features=384, out_features=144, bias=True)
        (1): CELU(alpha=0.1)
        (2): Linear(in_features=144, out_features=112, bias=True)
        (3): CELU(alpha=0.1)
        (4): Linear(in_features=112, out_features=96, bias=True)
        (5): CELU(alpha=0.1)
        (6): Linear(in_feat